# MicroStrategy Python API 

In [101]:
import platform
print('The code was written using\nPython Version: {}'.format(platform.python_version()))

The code was written using
Python Version: 3.6.4


### Import Libraries

In [57]:
import requests
import json
import pandas as pd
import numpy as np

<a id='top'></a>
## Resources:

MicroStrategy API references:
*  [MicroStrategy RESTful API Interactive (your Local Demo)](http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/api-docs/)
* [MicroStrategy RESTful API Interactive (external demo)](https://demo.microstrategy.com/MicroStrategyLibrary/api-docs/index.html#/)
* [MicroStrategy REST API Online Documentation](https://lw.microstrategy.com/msdz/MSDL/GARelease_Current/docs/projects/RESTSDK/Content/topics/REST_API/REST_API.htm)

Inspirations:
* [MicroStrategy Sample API Python Example by Robert Prochowicz](https://community.microstrategy.com/s/article/REST-API-10-9-code-example-in-Python)
* [Machine Learning with Python On-Demand Video with Scott Rigney](https://www.microstrategy.com/us/resources/library/webcasts/machine-learning-with-python-train-models-on-trust)

Python Library References:
* [Python Requests](http://docs.python-requests.org/en/master/)
* [Python JSON](https://docs.python.org/3/library/json.html)
* [Python Pandas](https://pandas.pydata.org/)

**List of Code Examples **:
1. [Authenticate User & Get Token](#auth)
2. [Logout User](#exit)
3. [Test Session](#test)
4. [Get User Info](#user)
5. [List Projects](#projects)
6. [Get Project Info](#projectinfo)
7. [List Objects](#objects)
8. [List Cubes/Reports](#listcubes)
9. [Search](#search)
10. [List Attributes/Metrics or Report/Cube](#reportobjects)
11. [Cube Instance](#cubeinstance)
12. [Get Cube Data](#cubedata)
13. [Create Cube](#createcube)
14. [Write to Cube](#writecube)

### Set Parameters 

In [58]:
### Parameters ###
username = 'Administrator'
password = ''
iserver = '10.254.113.99'
projectId = 'B19DEDCC11D4E0EFC000EB9495D0F44F'
projectName = 'MicroStrategy Tutorial'
baseURL = "http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/api/";

<a id='auth'></a>
## Authentication: return authToken & cookies
[Top](#top)

In [3]:
#### Get token ###
def login(baseURL,username,password):
    print("Getting token...")
    header = {'username': username,
                'password': password,
                'loginMode': 1}
    r = requests.post(baseURL + 'auth/login', data=header)
    if r.ok:
        authToken = r.headers['X-MSTR-AuthToken']
        cookies = dict(r.cookies)
        print("Token: " + authToken)
#         print(r.headers)
#         print(r.reason)
        return authToken, cookies
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))

In [277]:
authToken, cookies = login(baseURL,username,password)

Getting token...
Token: etsuff7t7g2hg0c1e17u891iq


In [278]:
print(cookies)
print(authToken)

{'JSESSIONID': '8486C94929AFE4A6774283A21E73081A'}
etsuff7t7g2hg0c1e17u891iq


<a id='test'></a>
## Test Session
[Top](#top)

In [6]:
def get_sessions(baseURL, auth_token, cookies):
    print("Checking session...")
    header = {'X-MSTR-AuthToken': auth_token,
                 'Accept': 'application/json'}
    r = requests.get(baseURL + "sessions", headers=header, cookies=cookies)
    
    if r.ok:
        print("Authenticated...")
        print(r)
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))
        print(r.url)
        print(r.content)
#         print(r.headers)
#         print(r.raw.connection)
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))
    

In [7]:
get_sessions(baseURL, authToken, cookies)

Checking session...
Authenticated...
<Response [200]>
HTTP 200 - OK, Message {"locale":1033,"maxSearch":3,"workingSet":10,"timeout":600,"id":"54F3D26011D2896560009A8E67019608","fullName":"Administrator","initials":"A"}
http://dub-tatwan10.corp.microstrategy.com:8080/MicroStrategyLibrary/api/sessions
b'{"locale":1033,"maxSearch":3,"workingSet":10,"timeout":600,"id":"54F3D26011D2896560009A8E67019608","fullName":"Administrator","initials":"A"}'


<a id='createcube'></a>
## Create Cube 
[Top](#top)

In [11]:
def create_cube(baseURL, auth_token, cookies, projectId, cube_structure_json):
    header = {'X-MSTR-AuthToken': auth_token,
                  'Content-Type': 'application/json',#IMPORTANT!
                  'Accept': 'application/json',
                  'X-MSTR-ProjectID': projectId}
    print("\nCreating new cube...")
    r = requests.post(baseURL + "datasets", headers=header, data=cube_structure_json, cookies=cookies)
    if r.ok:
        print("Error: " + str(r.raise_for_status()) + "   ||   HTTP Status Code: " + str(r.status_code))
        print("\nCube CREATED successfully")
        print("\nCube ID:     " + r.json()['datasetId'])
        print("Cube Name:   " + r.json()['name'])
        print("Table ID:    " + r.json()['tables'][0]['id'])
        print("Table Name:  " + r.json()['tables'][0]['name'])
        print("\nRemember to copy and note down Cube ID (dataset ID) and Table ID. Enter those values in the Python script 'Parameters' section")
        return (r.json()['tables'][0]['name'], r.json()['datasetId'] )
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))   

In [12]:
tablename, cubeid = create_cube(baseURL, authToken, cookies, projectId, cube_structure_json)


Creating new cube...
Error: None   ||   HTTP Status Code: 200

Cube CREATED successfully

Cube ID:     4E9123D6454616C82C6CAC80EA72202D
Cube Name:   SpeedCarCube1
Table ID:    D344304516503AC9B4157564E3F31869
Table Name:  Speed_Car_Table

Remember to copy and note down Cube ID (dataset ID) and Table ID. Enter those values in the Python script 'Parameters' section


<a id='writecube'></a>
## Write to Cube Example
[Top](#top)

In [13]:
def push_data(updatePolicy, pushed_data_json, auth_token, cookies, projectId, datasetId, tableId):
    pushed_data = base64.b64encode(bytes(pushed_data_json, 'utf-8')).decode('ascii')
    push_url = baseURL + "datasets/" + datasetId + "/tables/" + tableId
    header = {'X-MSTR-AuthToken': auth_token,
                    'Content-Type': 'application/json',#IMPORTANT!
                    'X-MSTR-ProjectID': projectId,
                    'updatePolicy': updatePolicy
                    }
    insert_data = ('{"name":"Speed_Car_Table",'
                   '"columnHeaders":['
                   '{"name": "Timestamp","dataType": "DATETIME"},'
                   '{"name": "Id_Run","dataType": "STRING"},'
                   '{"name": "Id_Measure_W","dataType": "DOUBLE"},'
                   '{"name": "Id_Measure","dataType": "DOUBLE"},'
                   '{"name": "Driver_Id","dataType": "DOUBLE"},'
                   '{"name": "Accel_X","dataType": "DOUBLE"},'
                   '{"name": "Accel_Y","dataType": "DOUBLE"},'
                   '{"name": "Accel_Z","dataType": "DOUBLE"},'
                   '{"name": "Gyro_X","dataType": "DOUBLE"},'
                   '{"name": "Gyro_Y","dataType": "DOUBLE"},'
                   '{"name": "Gyro_Z","dataType": "DOUBLE"},'
                   '{"name": "Humidity","dataType": "DOUBLE"},'
                   '{"name": "Pitch","dataType": "DOUBLE"},'
                   '{"name": "Pressure","dataType": "DOUBLE"},'
                   '{"name": "Roll","dataType": "DOUBLE"},'
                   '{"name": "Temp_H","dataType": "DOUBLE"},'
                   '{"name": "Yaw","dataType": "DOUBLE"}'
                   '],"data":"' + pushed_data + '"}'
                   )
    print("\nPushing data...")
    r = requests.patch(push_url, headers=header, data=insert_data, cookies=cookies)
    if r.ok:
        print("Pushed successfully...")
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))

In [14]:
sample_data = '[{"Timestamp": "10/5/2017 7:28:10 PM","Id_Run": "RESET","Id_Measure_W": 1,"Id_Measure": 1,"Driver_Id": 5678,"Accel_X": 0.02546,"Accel_Y": 0.09226,"Accel_Z": 0.97747,"Gyro_X": -0.0123,"Gyro_Y": 0.01709,"Gyro_Z": 0.00736,"Humidity": 38.81998,"Pitch": 358.58579,"Pressure": 980.43262,"Roll": 5.39072,"Temp_H": 34.27213,"Yaw": 67.82173},{"Timestamp": "10/5/2017 7:28:20 PM","Id_Run": "RESET","Id_Measure_W": 2,"Id_Measure": 2,"Driver_Id": 1234,"Accel_X": 0.09999,"Accel_Y": 0.09226,"Accel_Z": 0.97747,"Gyro_X": -0.0123,"Gyro_Y": 0.01709,"Gyro_Z": 0.00736,"Humidity": 38.81998,"Pitch": 358.58579,"Pressure": 980.43262,"Roll": 5.39072,"Temp_H": 34.27213,"Yaw": 67.82173}]'
push_data('Replace', sample_data, authToken, cookies, projectId, cubeid, tablename)


Pushing data...
Pushed successfully...
HTTP 200 - OK, Message 


In [15]:
print("token = {} and Cookies = {}".format(authToken, cookies))

token = mlfo8pbng5aq7u4fid9f9bpp3j and Cookies = {'JSESSIONID': '9237C6745F9DE0780F7220605495E1A8'}


<a id='user'></a>
## Get UserInfo
[Top](#top)

In [16]:
def user_info(baseURL, auth_token, cookies):
    print("Checking User Info...")
    header = {'X-MSTR-AuthToken': auth_token,
                 'Accept': 'application/json'}
    r = requests.get(baseURL + "sessions/userInfo", headers=header, cookies=cookies)
    
    if r.ok:
        print("Authenticated...")
        print(r)
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))
    

In [17]:
authToken, cookies = login(baseURL,username,password)

Getting token...
Token: q3u07klfe22f63jj77943mg53q


In [18]:
user_info(baseURL, authToken, cookies)

Checking User Info...
Authenticated...
<Response [200]>
HTTP 200 - OK, Message {"metadataUser":true,"id":"54F3D26011D2896560009A8E67019608","fullName":"Administrator","initials":"A"}


<a id='projectinfo'></a>
## Get Project Info
[Top](#top)

In [19]:
def projecT_Info(baseURL, authToken, projectId, cookies):
    print("Checking Project Info")
    header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                 'Accept': 'application/json'}
    
    r = requests.get(baseURL + 'sessions/projectId', headers=header, cookies=cookies)
    
    if r.ok:
        print("Success")
        print(r.status_code, r.reason, r.text)
    else:
        print(r.status_code, r.reason, r.text)

In [20]:
projecT_Info(baseURL, authToken, projectId, cookies)

Checking Project Info
Success
200 OK {"locale":1033,"maxSearch":3,"workingSet":10,"timeout":600,"id":"54F3D26011D2896560009A8E67019608","fullName":"Administrator","initials":"A"}


<a id='objects'></a>
## List Objects
[Top](#top)

In [21]:
def personal_objects(baseURL, authToken, projectId, cookies):
    print("Checking personal objects...")
    header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                 'Accept': 'application/json'}
    
    r = requests.get(baseURL + 'folders/myPersonalObjects', headers=header, cookies=cookies)
    
    if r.ok:
        print("Success")
        print(r.status_code, r.reason, r.text)
    else:
        print(r.status_code, r.reason, r.text)


In [22]:
personal_objects(baseURL, authToken, projectId, cookies)

Checking personal objects...
Success
200 OK [{"name":"My Answers","id":"8D67909611D3E4981000E787EC6DE8A4","type":8,"subtype":2048,"dateCreated":"2001-01-02T20:49:01.000+0000","dateModified":"2016-05-18T09:01:08.000+0000","version":"05A1BFB74FF80A51176360A7B788947B","acg":255,"owner":{"name":"Administrator","id":"54F3D26011D2896560009A8E67019608"}},{"name":"My Dossiers","id":"66C857944B1256F6BA4C62B740F14C70","type":8,"subtype":2048,"dateCreated":"2016-05-18T08:44:52.000+0000","dateModified":"2018-02-07T09:05:03.000+0000","version":"FC14F2724B610E24BD853DAD20CF65D3","acg":255,"owner":{"name":"Administrator","id":"54F3D26011D2896560009A8E67019608"}},{"name":"My Favorites","id":"8D67909A11D3E4981000E787EC6DE8A4","type":8,"subtype":2048,"dateCreated":"2001-01-02T20:49:01.000+0000","dateModified":"2016-05-18T09:01:08.000+0000","version":"615379844F4DB637C23D74B0EBF74D81","acg":255,"owner":{"name":"Administrator","id":"54F3D26011D2896560009A8E67019608"}},{"name":"My Objects","id":"8D67909E11

<a id='reportobjects'></a>
## List Cube Attributes and Metrics
[Top](#top)

In [23]:
def cubeInfo(baseURL, authToken, projectId, cookies):
    print("Checking personal objects...")
    header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                 'Accept': 'application/json'}
    
    r = requests.get(baseURL + 'cubes/' + 'BD23848347017FC2C0B4509AED1AF7B4', headers=header, cookies=cookies)
    
    if r.ok:
#         print("Success")
#         print(r.status_code, r.reason, r.text)
        return r
    else:
        print(r.status_code, r.reason, r.text)

In [24]:
c = cubeInfo(baseURL, authToken, projectId, cookies)

Checking personal objects...


In [27]:
jc = json.loads(c.text)
print('List of Attributes:')
for i in jc['result']['definition']['availableObjects']['attributes']:
    print(i['name'])

print('List of Metrics:')
for i in jc['result']['definition']['availableObjects']['metrics']:
    print(i['name'])


List of Attributes:
Country
Catalog
Category
Subcategory
List of Metrics:
Cost
Gross Revenue
Units Sold


In [179]:
node = c.json()
attr = node['result']['definition']['availableObjects']['attributes']
mtrcs = node['result']['definition']['availableObjects']['metrics']
reportObjects = pd.DataFrame(columns=('ObjectName', 'Type', 'ID'))
for i in attr:
    reportObjects = reportObjects.append(pd.DataFrame(data={'ObjectName': [i['name']], 'Type': [i['type']], 'ID': [i['id']]}), ignore_index=True)
for i in mtrcs:
    reportObjects = reportObjects.append(pd.DataFrame({'ObjectName': [i['name']], 'Type': [i['type']], 'ID': [i['id']]}), ignore_index=True)

attrNames = reportObjects[reportObjects['Type'] == 'Attribute'].iloc[:,1:2]
mtrcNames = reportObjects[reportObjects['Type'] == 'Metric'].iloc[:,1:2]

In [180]:
attribute_names = reportObjects[reportObjects['Type'] == 'Attribute']
attribute_names

,ID,ObjectName,Type
0,8D679D3811D3E4981000E787EC6DE8A4,Country,Attribute
1,8D679D3611D3E4981000E787EC6DE8A4,Catalog,Attribute
2,8D679D3711D3E4981000E787EC6DE8A4,Category,Attribute
3,8D679D4F11D3E4981000E787EC6DE8A4,Subcategory,Attribute


In [461]:
print(attrNames)
print(mtrcNames)

attrNames.shape, mtrcNames.shape

    ObjectName
0      Country
1      Catalog
2     Category
3  Subcategory
      ObjectName
4           Cost
5  Gross Revenue
6     Units Sold


((4, 1), (3, 1))

<a id='listcubes'></a>
## List Cubes or Reports
[Top](#top)

In [30]:
def cubeList(baseURL, authToken, projectId, cookies):
    print("Checking personal objects...")
    header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                 'Accept': 'application/json'}
    
    r = requests.get(baseURL + 'cubes', headers=header, cookies=cookies)
    
    if r.ok:
        print("Success")
        print(r.status_code, r.reason, r.text)
        print(json.loads(r.content))
    else:
        print(r.status_code, r.reason, r.text)


In [31]:
cubeList(baseURL, authToken, projectId, cookies)

Checking personal objects...
Success
200 OK {"cubesInfos":[]}
{'cubesInfos': []}


<a id='projects'></a>
## List of Projects
[Top](#top)

In [33]:
def listProjects(baseURL, authToken):   
    header = {'X-MSTR-AuthToken': authToken,
                 'Accept': 'application/json'}
    r = requests.get(baseURL + 'projects', headers=header, cookies=cookies)

    projectIDs = []
    projectNames = []
    for i in json.loads(r.text):
    #     print('Projet Name: {} , ID: {}'.format(i["name"], i["id"]))
        projectIDs.append(i["name"])
        projectNames.append(i["id"])
    projectList = pd.DataFrame(np.column_stack([projectIDs, projectNames]), columns=["ProjectName","ProjectID"]) 
    return projectList


In [34]:
listProjects(baseURL, authToken)

,ProjectName,ProjectID
0,MicroStrategy Tutorial,B19DEDCC11D4E0EFC000EB9495D0F44F
1,Human Resources Analysis Module,AF09B3E3458F78B4FBE4DEB68528BF7B
2,Enterprise Manager,4DD3B04B40D227471401609D630C76ED


In [35]:
x = listProjects(baseURL, authToken)
x.iloc[:,0]

0             MicroStrategy Tutorial
1    Human Resources Analysis Module
2                 Enterprise Manager
Name: ProjectName, dtype: object

<a id='search'></a>
## Search Objects
[Top](#top)

In [36]:
search_id = '6386CB974816A54B063EDF8833AD0EE1'
def search(baseURL, authToken, search_id):
    header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                  'Accept': 'application/json'}
#     searches/results?name=Customer&pattern=4&type=3&getAncestors=false&limit=-1&certifiedStatus=ALL
    
    r = requests.get(baseURL + 'searches/results?type=39', headers=header, cookies=cookies)
    
    if r.ok:
        a = json.loads(r.text)
        print("Success")
        searchName = []
        searchID = []

        for i in a['result']:
            searchName.append(i['name'])
            searchID.append(i['id'])
        searchList = pd.DataFrame(np.column_stack([searchName, searchID]), columns=["SearchName", "SearchID"]) 
        return searchList
    else:
        print(r.status_code, r.reason, r.text)
    

In [37]:
a = search(baseURL, authToken, search_id)

Success


In [39]:
a

,SearchName,SearchID
0,Search for all objects of type Grid,87F09D2EBB9B462CAC4581ABCAD97BBD
1,Search for all objects of type Text Prompt,8A7CAF697BB64191BA3E15FA10DEDA61
2,MicroStrategy Web User Objects,9F4A56074EDD734CBEFFC79A68BC36AF
3,Search for all objects of type Hierarchy,A1468ECD38754F90B56B611635AC550E
4,Search for all objects of type Column,57048C8A11D437E2C000039187BD3A4F
5,Search for all objects of type Document,57048CAE11D437E2C000039187BD3A4F
6,Search for all objects of type Custom Group,6E1876B4A5DE4EB1BBF68E0DC4F239E6
7,Search for all objects of type Logical Table,72E04CC0E35D415B9D2B6687B88A4699
8,Search for all objects of type HTML Document,7442C032B10546429F0559215D7CC5A1
9,Search for all objects of type Search,CEC5798322C34B1FAE017596B875BDFA


In [38]:
a.iloc[27:,1].values

array(['6386CB974816A54B063EDF8833AD0EE1'], dtype=object)

<a id='exit'></a>
##  Log Out
[Top](#top)

In [38]:
#### Get token ###
def logout(baseURL,authToken):
    
    header = {'X-MSTR-AuthToken': authToken,
                  'Accept': 'application/json'}

    r = requests.post(baseURL + 'auth/logout',headers=header, cookies=cookies)
    if r.ok:
        print("Logged Out")
       
    else:
        print("HTTP %i - %s, Message %s" % (r.status_code, r.reason, r.text))

In [48]:
logout(baseURL, authToken)

Logged Out


<a id='cubedata'></a>
## Get Data from a Cube into DataFrame
[Top](#top)

** Create Cube Instance **

In [40]:
authToken, cookies = login(baseURL, username, password)

cubeId = 'BD23848347017FC2C0B4509AED1AF7B4'

header = {'X-MSTR-AuthToken': authToken,
                  'X-MSTR-ProjectID': projectId,
                  'Accept': 'application/json'}

r = requests.post(baseURL + 'cubes/' + cubeId + '/instances?limit=1000', headers=header, cookies = cookies)
        
        

Getting token...
Token: gsn1bj73lh7cgbbdnruhkuick


In [41]:
cubeInstance_id = r.json()['instanceId']

Get cube data

In [42]:
r = requests.get(baseURL + 'cubes/' + cubeId + '/instances/' + cubeInstance_id + '?limit=1000', headers = header, cookies = cookies)

In [43]:
r.json()['result']['data']['paging']

{'current': 86,
 'limit': 1000,
 'next': None,
 'offset': 0,
 'prev': None,
 'total': 86}

In [395]:
response = r.json()['result']['data']['root']

In [396]:
parsed_rows = r.json()['result']['data']['root']

In [397]:
metrs = []
attrs = []
temp = [None] * len(attribute_names)

for e in parsed_rows:
    print(e)

isPartial
children


In [433]:
_cubeId = r.json()['id']
_cubeInstanceId = r.json()['instanceId']
_cubeName = r.json()['name']
print("Cube Info \nCube ID: \t{}\nCube Instance:\t{}\nCube Name:\t{}".format(_cubeId, cubeInstance_id, _cubeName))

totalRecords = r.json()['result']['data']['paging']['total']
print('Number of Rows returned: {}'.format(totalRecords))

Cube Info 
Cube ID: 	BD23848347017FC2C0B4509AED1AF7B4
Cube Instance:	C23925BA421DE47033948091404CCE06
Cube Name:	Cube converted from Summer Sales by Country per Catalog
Number of Rows returned: 86


In [399]:
pd.DataFrame.from_dict(response, orient='columns')
from pandas.io.json import json_normalize
pd.DataFrame.from_dict(json_normalize(response['children'][0]['children'][0]['children'][0]['children']))

,depth,element.attributeIndex,element.formValues.DESC,element.id,element.name,metrics.Cost.fv,metrics.Cost.mi,metrics.Cost.rv,metrics.Gross Revenue.fv,metrics.Gross Revenue.mi,metrics.Gross Revenue.rv,metrics.Units Sold.fv,metrics.Units Sold.mi,metrics.Units Sold.rv
0,3,3,Art & Architecture,h11;8D679D4F11D3E4981000E787EC6DE8A4,Art & Architecture,"$18,010",0,18010.499,"$24,282",1,24282,"1,349",2,1349
1,3,3,Business,h12;8D679D4F11D3E4981000E787EC6DE8A4,Business,"$23,180",0,23180.417,"$33,199",1,33199,"2,657",2,2657
2,3,3,Literature,h13;8D679D4F11D3E4981000E787EC6DE8A4,Literature,"$82,955",0,82954.898,"$107,529",1,107529,"14,307",2,14307
3,3,3,Books - Miscellaneous,h14;8D679D4F11D3E4981000E787EC6DE8A4,Books - Miscellaneous,"$91,711",0,91711.100,"$112,440",1,112440,"12,284",2,12284
4,3,3,Science & Technology,h15;8D679D4F11D3E4981000E787EC6DE8A4,Science & Technology,"$62,821",0,62820.563,"$83,944",1,83944,"3,235",2,3235


In [400]:
for i in response['children']:
    print(i['element']['attributeIndex'], i['element']['name'])
    for j in i['children']:
        print('\t',j['element']['attributeIndex'], j['element']['name'])
        for k in j['children']:
            print('\t\t',k['element']['attributeIndex'], k['element']['name'])
            for y in k['children']:
                print('\t\t\t',y['element']['attributeIndex'], y['element']['name'])

0 Summer 2011
	 1 Books
		 2 USA
			 3 Art & Architecture
			 3 Business
			 3 Literature
			 3 Books - Miscellaneous
			 3 Science & Technology
		 2 Web
			 3 Art & Architecture
			 3 Business
			 3 Literature
			 3 Books - Miscellaneous
			 3 Science & Technology
	 1 Electronics
		 2 USA
			 3 Audio Equipment
			 3 Cameras
			 3 Computers
			 3 Electronics - Miscellaneous
			 3 TV's
			 3 Video Equipment
		 2 Web
			 3 Audio Equipment
			 3 Cameras
			 3 Computers
			 3 Electronics - Miscellaneous
			 3 TV's
			 3 Video Equipment
	 1 Movies
		 2 USA
			 3 Action
			 3 Comedy
			 3 Drama
			 3 Horror
			 3 Kids / Family
			 3 Special Interests
		 2 Web
			 3 Action
			 3 Comedy
			 3 Drama
			 3 Horror
			 3 Kids / Family
			 3 Special Interests
	 1 Music
		 2 USA
			 3 Country
			 3 Music - Miscellaneous
			 3 Pop
			 3 Rock
			 3 Soul / R&B
		 2 Web
			 3 Country
			 3 Music - Miscellaneous
			 3 Pop
			 3 Rock
			 3 Soul / R&B
0 Summer 2012
	 1 Books
		 2 USA
			 3 Business
			 3 L

In [425]:
#dd = []
def parseThroughHierarchy(response):
    dd = []
    if 'children' in response.keys():
        tmp = []
        for i in response['children']:
            dd.append((i['element']['attributeIndex'], i['element']['name']))
            dd += parseThroughHierarchy(i)
    else:
        tmp = []
        for i in response['metrics'].values():
            tmp.append(i['rv'])
        dd.append(tmp)
    return dd


tmp = [None] * len(attrNames)
for items in  parseThroughHierarchy(response):
    if isinstance(items, tuple):
        attrIndex, attrName = items 
        tmp[attrIndex] = attrName
        if attrIndex == len(attrNames)-1:
            attributesCapture.append(tmp[:])
    elif isinstance(element, list):
        metricsCapture.append(items)

In [452]:
ddf = []
for a, m in zip(attributesCapture, metricsCapture):
    ddf.append(a+m)
pd.DataFrame(ddf)

array([['Country'],
       ['Catalog'],
       ['Category'],
       ['Subcategory'],
       ['Cost'],
       ['Gross Revenue'],
       ['Units Sold']], dtype=object)

In [460]:
zero_data = np.zeros(shape=(totalRecords, len(mtrcNames)+len(attrNames)))
df = pd.DataFrame(zero_data, columns= pd.concat([attrNames, mtrcNames]))
df

,"(Country,)","(Catalog,)","(Category,)","(Subcategory,)","(Cost,)","(Gross Revenue,)","(Units Sold,)"
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
